# Sentiment Analysis - World News Subreddit

In [267]:
%%configure -f
{"executorMemory": "2G","driverMemory":"1G","executorCores": 1,"numExecutors": 3, 
 "conf": {"spark.dynamicAllocation.enabled": "false", 
          "spark.sql.parquet.enableVectorizedReader": "false", 
          "spark.pyspark.python": "python3",
          "spark.pyspark.virtualenv.enabled": "true",
          "spark.pyspark.virtualenv.type": "native",
          "spark.pyspark.virtualenv.bin.path": "/usr/bin/virtualenv"}}



Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
324,application_1681238170162_0324,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
292,application_1681238170162_0292,pyspark,idle,Link,Link,None,
312,application_1681238170162_0312,pyspark,idle,Link,Link,None,
321,application_1681238170162_0321,pyspark,idle,Link,Link,None,
324,application_1681238170162_0324,pyspark,idle,Link,Link,None,✔


In [268]:
%%spark

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [226]:
sc.addPyFile('s3://msbx5420-spr23/class/spark-nlp_2.12-4.4.1.jar')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [271]:
sc.list_packages()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Package                    Version
-------------------------- ------------------
absl-py                    1.3.0
astor                      file-astor-VERSION
astunparse                 1.6.3
aws-cfn-bootstrap          2.0
beautifulsoup4             4.9.3
bleach                     3.3.1
boto                       2.49.0
cachetools                 4.1.1
certifi                    2022.12.7
chardet                    5.1.0
charset-normalizer         3.0.1
click                      8.1.3
cryptography               3.2.1
cycler                     0.11.0
docutils                   0.14
flatbuffers                22.12.6
fonttools                  4.38.0
funcsigs                   1.0.2
gast                       0.5.3
google-auth                1.23.0
google-auth-oauthlib       0.4.2
google-pasta               0.2.0
grpcio                     1.29.0
h5py                       2.10.0
html5lib                   1.1
idna                       2.10
importlib-metadata         2.0.0
jmespath 

In [270]:
sc.install_pypi_package("pandas")
sc.install_pypi_package("matplotlib")
sc.install_pypi_package("wordcloud")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [144]:
#%%local

#from pyspark import SparkContext
#sc = SparkContext()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [229]:
import pandas as pd
pd.set_option('max_colwidth', 800)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [230]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
        .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.12:4.3.2") \
        .getOrCreate()

spark


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…



Read multiple files in a dir as one Dataframe


In [231]:
# read a CSV file into a Spark DataFrame
df = spark.read.csv("s3://msbx5420-spr23/Group_2/reddit_worldnews_start_to_2016-11-22.csv", header=True, inferSchema=True)

# show the first few rows of the DataFrame
df.show(5)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+------------+--------+----------+--------------------+-------+--------+---------+
|time_created|date_created|up_votes|down_votes|               title|over_18|  author|subreddit|
+------------+------------+--------+----------+--------------------+-------+--------+---------+
|  1201232046|  2008-01-25|       3|         0|Scores killed in ...|  False|   polar|worldnews|
|  1201232075|  2008-01-25|       2|         0|Japan resumes ref...|  False|   polar|worldnews|
|  1201232523|  2008-01-25|       3|         0|US presses Egypt ...|  False|   polar|worldnews|
|  1201233290|  2008-01-25|       1|         0|Jump-start econom...|  False| fadi420|worldnews|
|  1201274720|  2008-01-25|       4|         0|Council of Europe...|  False|mhermans|worldnews|
+------------+------------+--------+----------+--------------------+-------+--------+---------+
only showing top 5 rows

In [232]:
print(df.count())
df.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

520310
root
 |-- time_created: string (nullable = true)
 |-- date_created: string (nullable = true)
 |-- up_votes: string (nullable = true)
 |-- down_votes: string (nullable = true)
 |-- title: string (nullable = true)
 |-- over_18: string (nullable = true)
 |-- author: string (nullable = true)
 |-- subreddit: string (nullable = true)

In [233]:
from pyspark.sql.functions import col

dfAuthorTitle = df.select(col("title"), col("author"))
dfAuthorTitle.limit(5).toPandas()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

                                             title    author
0                Scores killed in Pakistan clashes     polar
1                 Japan resumes refuelling mission     polar
2                  US presses Egypt on Gaza border     polar
3     Jump-start economy: Give health care to all    fadi420
4  Council of Europe bashes EU&UN terror blacklist  mhermans

# Part of Speech Tagging (POS)


Try to implement the equivalent of flatMap in dataframes


In [234]:
import pyspark.sql.functions as F

dfWordCount = df.select(F.explode(F.split(df.title, "\\s+")).alias("word")) \
                .groupBy("word").count() \
                .orderBy(F.desc("count"))

dfWordCount.limit(10).toPandas()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  word   count
0   to  202905
1   in  194387
2  the  173461
3   of  168314
4    a   91871
5  and   83186
6    s   77617
7  for   75286
8   on   69446
9        54419

In [235]:
#pip install spark-nlp==4.4.0
sc.install_pypi_package("spark-nlp==4.4.0")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…


Use an NLP libary to do Part-of-Speech Tagging


In [184]:
# Import necessary libraries
from sparknlp.base import DocumentAssembler
from sparknlp.annotator import Tokenizer, LemmatizerModel, StopWordsCleaner, PerceptronModel, SentimentDetector
from pyspark.sql.functions import explode, col
from pyspark.sql.types import StringType
from pyspark.ml import Pipeline
from pyspark.sql.functions import desc
from sparknlp.pretrained import PretrainedPipeline

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [185]:
# Initialize Spark NLP
import sparknlp
spark = sparknlp.start()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [186]:
# Load the pre-trained POS tagger
pos_model = PerceptronModel.pretrained()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

pos_anc download started this may take some time.
Approximate size to download 3.9 MB
[OK!]

In [118]:
# Load the pre-trained sentiment analysis model
#pipeline1 = PretrainedPipeline('analyze_sentiment', lang='en')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [187]:
# Define the Spark DataFrame with the text to annotate
text_df = dfAuthorTitle.selectExpr("title as text")
text_df.limit(5).toPandas()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

                                              text
0                Scores killed in Pakistan clashes
1                 Japan resumes refuelling mission
2                  US presses Egypt on Gaza border
3     Jump-start economy: Give health care to all 
4  Council of Europe bashes EU&UN terror blacklist

In [188]:
# Create the DocumentAssembler and annotate the text
documentAssembler = DocumentAssembler().setInputCol("text").setOutputCol("document")
text_df = documentAssembler.transform(text_df)
tokenizer = Tokenizer().setInputCols(["document"]).setOutputCol("token")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [189]:
# Remove stop words and lemmatize the text
stopWordsCleaner = StopWordsCleaner().setInputCols(["token"]).setOutputCol("cleanTokens")
lemmatizer = LemmatizerModel.pretrained().setInputCols(["cleanTokens"]).setOutputCol("lemma")


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

lemma_antbnc download started this may take some time.
Approximate size to download 907.6 KB
[OK!]

In [190]:
# Add POS tagging to the text
posTagger = PerceptronModel.pretrained().setInputCols(["lemma", "document"]).setOutputCol("pos")


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

pos_anc download started this may take some time.
Approximate size to download 3.9 MB
[OK!]

In [191]:
# Assemble the pipeline
pipeline = Pipeline(stages=[documentAssembler, tokenizer, stopWordsCleaner, lemmatizer, posTagger])


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [192]:
# Fit the pipeline to the DataFrame and transform the DataFrame
text_df = pipeline.fit(text_df).transform(text_df)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [193]:
# Show the annotated DataFrame
text_df.limit(5).toPandas()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

                                              text  ...                                                                                                                                                                                                                                                                                                                                                                         pos
0                Scores killed in Pakistan clashes  ...                                                                                                                              [(pos, 0, 5, NNP, {'sentence': '0', 'word': 'Scores'}, []), (pos, 7, 12, VB, {'sentence': '0', 'word': 'kill'}, []), (pos, 17, 24, NNP, {'sentence': '0', 'word': 'Pakistan'}, []), (pos, 26, 32, NN, {'sentence': '0', 'word': 'clash'}, [])]
1                 Japan resumes refuelling mission  ...                                                                                                         

In [194]:
# Extract proper nouns (NNP and NNPS) from the POS tagged text
dfPos = text_df.select(explode("pos").alias("pos"))
nnpFilter = "pos.result = 'NNP' or pos.result = 'NNPS' "
dfNNP = dfPos.where(nnpFilter)
dfWordTag = dfNNP.selectExpr("pos.metadata['word'] as word", "pos.result as tag")


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [200]:
# Show the most frequent proper nouns
from pyspark.sql.functions import desc
dfWordTag.groupBy("word").count().orderBy(desc("count")).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+-----+
|     word|count|
+---------+-----+
|    China|24243|
|       US|21757|
|    Syria|18127|
|   Russia|17304|
|      U.S|16007|
|   Israel|14800|
|     Iran|14714|
|       UK|11140|
|    Korea|10982|
|    South|10419|
|    North|10295|
|  Ukraine| 9995|
|    India| 9984|
|      New| 9274|
|President| 8681|
|       UN| 8624|
|    World| 8230|
|       EU| 8168|
|     News| 8152|
|    Japan| 8000|
+---------+-----+
only showing top 20 rows

In [ ]:
#pip install wordcloud

In [214]:
# Generate word cloud visualization
from wordcloud import WordCloud
import matplotlib.pyplot as plt


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Word Cloud based on general news sentiment

In [215]:
# Convert the DataFrame to a list of words
words_list = [row.word for row in dfWordTag.collect()]

# Generate the word cloud
wordcloud = WordCloud(width = 800, height = 800, background_color ='white', min_font_size = 10).generate(' '.join(words_list))

# Plot the word cloud
plt.figure(figsize = (8, 8), facecolor = None)
plt.imshow(wordcloud)
plt.axis("off")
plt.tight_layout(pad = 0)
  
# Show the plot
plt.show()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Sentiment Analysis

In [265]:
%%configure -f
{"executorMemory": "2G","driverMemory":"1G","executorCores": 1,"numExecutors": 3, 
 "conf": {"spark.dynamicAllocation.enabled": "false", 
          "spark.sql.parquet.enableVectorizedReader": "false", 
          "spark.pyspark.python": "python3",
          "spark.pyspark.virtualenv.enabled": "false",
          "spark.pyspark.virtualenv.type": "native",
          "spark.pyspark.virtualenv.bin.path": "/usr/bin/virtualenv"}}

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
323,application_1681238170162_0323,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
292,application_1681238170162_0292,pyspark,idle,Link,Link,None,
312,application_1681238170162_0312,pyspark,idle,Link,Link,None,
321,application_1681238170162_0321,pyspark,idle,Link,Link,None,
323,application_1681238170162_0323,pyspark,idle,Link,Link,None,✔


In [247]:
# Import necessary libraries
from pyspark.ml import PipelineModel
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
#from sparknlp.base import *
#from sparknlp.annotator import *
from pyspark.ml import Pipeline
from pyspark.sql.functions import length

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [273]:
# read a CSV file into a Spark DataFrame
df = spark.read.csv("s3://msbx5420-spr23/Group_2/reddit_worldnews_start_to_2016-11-22.csv", header=True, inferSchema=True)

df.cache()
# show the first few rows of the DataFrame
df.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+------------+--------+----------+--------------------+-------+--------+---------+
|time_created|date_created|up_votes|down_votes|               title|over_18|  author|subreddit|
+------------+------------+--------+----------+--------------------+-------+--------+---------+
|  1201232046|  2008-01-25|       3|         0|Scores killed in ...|  False|   polar|worldnews|
|  1201232075|  2008-01-25|       2|         0|Japan resumes ref...|  False|   polar|worldnews|
|  1201232523|  2008-01-25|       3|         0|US presses Egypt ...|  False|   polar|worldnews|
|  1201233290|  2008-01-25|       1|         0|Jump-start econom...|  False| fadi420|worldnews|
|  1201274720|  2008-01-25|       4|         0|Council of Europe...|  False|mhermans|worldnews|
+------------+------------+--------+----------+--------------------+-------+--------+---------+
only showing top 5 rows

In [250]:
#import nltk
#nltk.download('vader_lexicon')
import pyspark.sql.functions as F
from pyspark.sql.functions import col

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [251]:
#Check for duplicates
counts = df.groupby(df.time_created).count()
duplicates = counts.filter(col("count") > 1)
duplicates.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+-----+
|time_created|count|
+------------+-----+
|  1415800214|    2|
|  1371521658|    2|
|  1406022773|    2|
|  1386162933|    2|
|  1407782198|    2|
|  1414496883|    2|
|  1411567558|    2|
|  1411553994|    2|
|  1406800669|    2|
|  1369036843|    2|
|  1410950940|    2|
|  1380303073|    2|
|  1391019354|    2|
|  1409813706|    2|
|  1418134027|    2|
|  1399283088|    2|
|  1408395545|    2|
|  1369331524|    2|
|  1391442017|    2|
|  1457241470|    2|
+------------+-----+
only showing top 20 rows

In [253]:
#Count of duplicates
df.createOrReplaceTempView("data")
print(spark.sql("SELECT count(DISTINCT time_created) FROM data").show())
spark.sql("SELECT count(time_created) FROM data").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------------------+
|count(DISTINCT time_created)|
+----------------------------+
|                      513773|
+----------------------------+

None
+-------------------+
|count(time_created)|
+-------------------+
|             520310|
+-------------------+

In [254]:
#Create identifier unique_ID
from pyspark.sql.functions import monotonically_increasing_id
new_df = df.withColumn("unique_ID", monotonically_increasing_id())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [255]:
#Show new_df, which is the original df with unique identifiers
new_df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+------------+--------+----------+--------------------+-------+--------------+---------+---------+
|time_created|date_created|up_votes|down_votes|               title|over_18|        author|subreddit|unique_ID|
+------------+------------+--------+----------+--------------------+-------+--------------+---------+---------+
|  1201232046|  2008-01-25|       3|         0|Scores killed in ...|  False|         polar|worldnews|        0|
|  1201232075|  2008-01-25|       2|         0|Japan resumes ref...|  False|         polar|worldnews|        1|
|  1201232523|  2008-01-25|       3|         0|US presses Egypt ...|  False|         polar|worldnews|        2|
|  1201233290|  2008-01-25|       1|         0|Jump-start econom...|  False|       fadi420|worldnews|        3|
|  1201274720|  2008-01-25|       4|         0|Council of Europe...|  False|      mhermans|worldnews|        4|
|  1201287889|  2008-01-25|      15|         0|Hay presto! Farme...|  False|Armagedonovich|worldnews|   

In [256]:
#Creating a copy of new_df and storing it as df for further analysis
df = new_df.alias('df')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [258]:
#Isolate relevant columns into new DF
new_new_df = new_df.select("unique_ID","title")
new_new_df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+--------------------+
|unique_ID|               title|
+---------+--------------------+
|        0|Scores killed in ...|
|        1|Japan resumes ref...|
|        2|US presses Egypt ...|
|        3|Jump-start econom...|
|        4|Council of Europe...|
|        5|Hay presto! Farme...|
|        6|Strikes, Protests...|
|        7|The U.N. Mismanag...|
|        8|Nicolas Sarkozy t...|
|        9|US plans for miss...|
|       10|Archbishop of Can...|
|       11|Top US Envoy: Vio...|
|       12|Team building flo...|
|       13|Migrant workers t...|
|       14| Sarkozy, Girlfri...|
|       15|Nicolas Sarkozy, ...|
|       16|Mass Evacuations ...|
|       17|Poor Haitians Res...|
|       18|European Commissi...|
|       19|Rambo banned in B...|
+---------+--------------------+
only showing top 20 rows

In [259]:
sc.addFile("s3://msbx5420-spr23/class/nltk_data/", True)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [260]:
import nltk
from pyspark import SparkFiles
from nltk.sentiment import SentimentIntensityAnalyzer


# Define a UDF to find the sentiment of a title
def find_sentiment(title):
    nltk.data.path.append(SparkFiles.getRootDirectory() + '/nltk_data')
    
    # Create a SentimentIntensityAnalyzer object
    sia = SentimentIntensityAnalyzer()

    if title is None:
        return "Neutral"
    if sia.polarity_scores(title)["compound"] > 0:
        return "Positive"
    elif sia.polarity_scores(title)["compound"] < 0:
        return "Negative"
    else:
        return "Neutral"

# Register the UDF
find_sentiment_udf = udf(find_sentiment, StringType())

# Add a new column "title_sentiment" to the existing DataFrame "df"
df = df.withColumn("title_sentiment", find_sentiment_udf("title"))

df.cache()

# Show the result
df.show()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+------------+--------+----------+--------------------+-------+--------------+---------+---------+---------------+
|time_created|date_created|up_votes|down_votes|               title|over_18|        author|subreddit|unique_ID|title_sentiment|
+------------+------------+--------+----------+--------------------+-------+--------------+---------+---------+---------------+
|  1201232046|  2008-01-25|       3|         0|Scores killed in ...|  False|         polar|worldnews|        0|       Negative|
|  1201232075|  2008-01-25|       2|         0|Japan resumes ref...|  False|         polar|worldnews|        1|        Neutral|
|  1201232523|  2008-01-25|       3|         0|US presses Egypt ...|  False|         polar|worldnews|        2|        Neutral|
|  1201233290|  2008-01-25|       1|         0|Jump-start econom...|  False|       fadi420|worldnews|        3|       Positive|
|  1201274720|  2008-01-25|       4|         0|Council of Europe...|  False|      mhermans|worldnews|   

In [275]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# Create three different DataFrames based on the sentiment values
df_positive = df.filter(df.title_sentiment == 'Positive')
df_negative = df.filter(df.title_sentiment == 'Negative')
df_neutral = df.filter((df.title_sentiment == 'Neutral') & (df.title.isNotNull())) 

# Concatenate the titles from each DataFrame into three separate strings
positive_titles = ' '.join(df_positive.select('title').rdd.flatMap(lambda x: x).collect())
negative_titles = ' '.join(df_negative.select('title').rdd.flatMap(lambda x: x).collect())
neutral_titles = ' '.join(df_neutral.select('title').rdd.flatMap(lambda x: x).collect())


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
#To exclude countries and nationalities
# Define lists of countries and nationalities
countries = ['Afghanistan', 'Aland Islands', 'Albania', 'Algeria', 'American Samoa', 'Andorra', 'Angola', 'Anguilla', 'Antarctica', 'Antigua and Barbuda', 'Argentina', 'Armenia', 'Aruba', 'Australia', 'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain', 'Bangladesh', 'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin', 'Bermuda', 'Bhutan', 'Bolivia, Plurinational State of', 'Bonaire, Sint Eustatius and Saba', 'Bosnia and Herzegovina', 'Botswana', 'Bouvet Island', 'Brazil', 'British Indian Ocean Territory', 'Brunei Darussalam', 'Bulgaria', 'Burkina Faso', 'Burundi', 'Cambodia', 'Cameroon', 'Canada', 'Cape Verde', 'Cayman Islands', 'Central African Republic', 'Chad', 'Chile', 'China', 'Christmas Island', 'Cocos (Keeling) Islands', 'Colombia', 'Comoros', 'Congo', 'Congo, The Democratic Republic of the', 'Cook Islands', 'Costa Rica', "Côte d'Ivoire", 'Croatia', 'Cuba', 'Curaçao', 'Cyprus', 'Czech Republic', 'Denmark', 'Djibouti', 'Dominica', 'Dominican Republic', 'Ecuador', 'Egypt', 'El Salvador', 'Equatorial Guinea', 'Eritrea', 'Estonia', 'Ethiopia', 'Falkland Islands (Malvinas)', 'Faroe Islands', 'Fiji', 'Finland', 'France', 'French Guiana', 'French Polynesia', 'French Southern Territories', 'Gabon', 'Gambia', 'Georgia', 'Germany', 'Ghana', 'Gibraltar', 'Greece', 'Greenland', 'Grenada', 'Guadeloupe', 'Guam', 'Guatemala', 'Guernsey', 'Guinea', 'Guinea-Bissau', 'Guyana', 'Haiti', 'Heard Island and McDonald Islands', 'Holy See (Vatican City State)', 'Honduras', 'Hong Kong', 'Hungary', 'Iceland', 'India', 'Indonesia', 'Iran, Islamic Republic of', 'Iraq', 'Ireland', 'Isle of Man', 'Israel', 'Italy', 'Jamaica', 'Japan', 'Jersey', 'Jordan', 'Kazakhstan', 'Kenya', 'Kiribati', "Korea, Democratic People's Republic of", 'Korea, Republic of', 'Kuwait', 'Kyrgyzstan', "Lao People's Democratic Republic", 'Latvia', 'Lebanon', 'Lesotho', 'Liberia', 'Libya', 'Liechtenstein', 'Lithuania', 'Luxembourg', 'Macao', 'Macedonia, Republic of', 'Madagascar', 'Malawi', 'Malaysia', 'Maldives', 'Mali', 'Malta', 'Marshall Islands', 'Martinique', 'Mauritania', 'Mauritius', 'Mayotte', 'Mexico', 'Micronesia, Federated States of', 'Moldova, Republic of', 'Monaco', 'Mongolia', 'Montenegro', 'Montserrat', 'Morocco', 'Mozambique', 'Myanmar', 'Namibia', 'Nauru', 'Nepal', 'Netherlands', 'New Caledonia', 'New Zealand', 'Nicaragua', 'Niger', 'Nigeria', 'Niue', 'Norfolk Island', 'Northern Mariana Islands', 'Norway', 'Oman', 'Pakistan', 'Palau', 'Palestinian Territory, Occupied', 'Panama', 'Papua New Guinea', 'Paraguay', 'Peru', 'Philippines', 'Pitcairn', 'Poland', 'Portugal', 'Puerto Rico', 'Qatar', 'Réunion', 'Romania', 'Russian Federation', 'Rwanda', 'Saint Barthélemy', 'Saint Helena, Ascension and Tristan da Cunha', 'Saint Kitts and Nevis', 'Saint Lucia', 'Saint Martin (French part)', 'Saint Pierre and Miquelon', 'Saint Vincent and the Grenadines', 'Samoa', 'San Marino', 'Sao Tome and Principe', 'Saudi Arabia', 'Senegal', 'Serbia', 'Seychelles', 'Sierra Leone', 'Singapore', 'Sint Maarten (Dutch part)', 'Slovakia', 'Slovenia', 'Solomon Islands', 'Somalia', 'South Africa', 'South Georgia and the South Sandwich Islands', 'Spain', 'Sri Lanka', 'Sudan', 'Suriname', 'South Sudan', 'Svalbard and Jan Mayen', 'Swaziland', 'Sweden', 'Switzerland', 'Syrian Arab Republic', 'Taiwan, Province of China', 'Tajikistan', 'Tanzania, United Republic of', 'Thailand', 'Timor-Leste', 'Togo', 'Tokelau', 'Tonga', 'Trinidad and Tobago', 'Tunisia', 'Turkey', 'Turkmenistan', 'Turks and Caicos Islands', 'Tuvalu', 'Uganda', 'Ukraine', 'United Arab Emirates', 'United Kingdom', 'United States', 'United States Minor Outlying Islands', 'Uruguay', 'Uzbekistan', 'Vanuatu', 'Venezuela, Bolivarian Republic of', 'Viet Nam', 'Virgin Islands, British', 'Virgin Islands, U.S.', 'Wallis and Futuna', 'Yemen', 'Zambia', 'Zimbabwe', 'Islamic State']

nationalities = ['Afghan', 'Albanian', 'Algerian', 'American', 'Andorran', 'Angolan', 'Antiguans', 'Argentinean', 'Armenian', 'Australian', 'Austrian', 'Azerbaijani', 'Bahamian', 'Bahraini', 'Bangladeshi', 'Barbadian', 'Barbudans', 'Batswana', 'Belarusian', 'Belgian', 'Belizean', 'Beninese', 'Bhutanese', 'Bolivian', 'Bosnian', 'Brazilian', 'British', 'Bruneian', 'Bulgarian', 'Burkinabe', 'Burmese', 'Burundian', 'Cambodian', 'Cameroonian', 'Canadian', 'Cape Verdean', 'Central African', 'Chadian', 'Chilean', 'Chinese', 'Colombian', 'Comoran',  'Congolese', 'Costa Rican', 'Croatian', 'Cuban', 'Cypriot', 'Czech', 'Danish', 'Djibouti', 'Dominican', 'Dutch', 'Dutchman', 'Dutchwoman', 'East Timorese', 'Ecuadorean', 'Egyptian', 'Emirian', 'Equatorial Guinean', 'Eritrean', 'Estonian', 'Ethiopian', 'Fijian', 'Filipino', 'Finnish', 'French', 'Gabonese', 'Gambian', 'Georgian', 'German', 'Ghanaian', 'Greek', 'Grenadian', 'Guatemalan', 'Guinea-Bissauan', 'Guinean', 'Guyanese', 'Haitian', 'Herzegovinian', 'Honduran', 'Hungarian', 'I-Kiribati', 'Icelander', 'Indian', 'Indonesian', 'Iranian', 'Iraqi', 'Irish', 'Israeli', 'Italian', 'Ivorian', 'Jamaican', 'Japanese', 'Jordanian', 'Kazakhstani', 'Kenyan', 'Kittian and Nevisian', 'Kuwaiti', 'Kyrgyz', 'Laotian', 'Latvian', 'Lebanese', 'Liberian', 'Libyan', 'Liechtensteiner', 'Lithuanian', 'Luxembourger', 'Macedonian', 'Malagasy', 'Malawian', 'Malaysian', 'Maldivan', 'Malian', 'Maltese', 'Marshallese', 'Mauritanian', 'Mauritian', 'Mexican', 'Micronesian', 'Moldovan', 'Monacan', 'Mongolian', 'Moroccan', 'Mosotho', 'Motswana', 'Mozambican', 'Namibian', 'Nauruan', 'Nepalese', 'Netherlander', 'New Zealander', 'Ni-Vanuatu', 'Nicaraguan', 'Nigerian', 'Nigerien', 'North Korean', 'Northern Irish', 'Norwegian', 'Omani', 'Pakistani', 'Palauan', 'Panamanian', 'Papua New Guinean', 'Paraguayan', 'Peruvian', 'Polish', 'Portuguese', 'Qatari', 'Romanian', 'Russian', 'Rwandan', 'Saint Lucian', 'Salvadoran', 'Samoan', 'San Marinese', 'Sao Tomean', 'Saudi', 'Scottish', 'Senegalese', 'Serbian', 'Seychellois', 'Sierra Leonean', 'Singaporean', 'Slovakian', 'Slovenian', 'Solomon Islander', 'Somali', 'South African', 'South Korean', 'Spanish', 'Sri Lankan', 'Sudanese', 'Surinamer', 'Swazi', 'Swedish', 'Swiss', 'Syrian', 'Taiwanese', 'Tajik', 'Tanzanian', 'Thai', 'Togolese', 'Tongan', 'Trinidadian or Tobagonian', 'Tunisian', 'Turkish', 'Tuvaluan', 'Ugandan', 'Ukrainian', 'Uruguayan', 'Uzbekistani', 'Venezuelan', 'Vietnamese', 'Welsh', 'Yemenite', 'Zambian', 'Zimbabwean']


# Filter out countries and nationalities from each string
for word_list in [countries, nationalities]:
    for word in word_list:
        positive_titles = positive_titles.replace(word, '')
        negative_titles = negative_titles.replace(word, '')
        neutral_titles = neutral_titles.replace(word, '')


In [ ]:
# Generate word clouds for each sentiment category
wordcloud_positive = WordCloud(width = 800, height = 800,background_color="white", min_font_size = 10).generate(positive_titles)
wordcloud_negative = WordCloud(width = 800, height = 800,background_color="white", min_font_size = 10).generate(negative_titles)
wordcloud_neutral = WordCloud(width = 800, height = 800,background_color="white", min_font_size = 10).generate(neutral_titles)

Word Cloud for Positive Sentiment

In [ ]:
# Display the word cloud for positive sentiment
plt.figure(figsize = (8, 8), facecolor = None)
plt.imshow(wordcloud_positive, interpolation='bilinear')
plt.axis("off")
plt.tight_layout(pad = 0)
plt.title("Positive Sentiments")
plt.show()


Word Cloud for Negative Sentiment

In [ ]:
# Display the word cloud for negative sentiment
plt.figure(figsize = (8, 8), facecolor = None)
plt.imshow(wordcloud_negative, interpolation='bilinear')
plt.axis("off")
plt.title("Negative Sentiments")
plt.tight_layout(pad = 0)
plt.show()

Word Cloud for Neutral Sentiment

In [ ]:
# Display the word cloud for neutral sentiment
plt.figure(figsize = (8, 8), facecolor = None)
plt.imshow(wordcloud_neutral, interpolation='bilinear')
plt.axis("off")
plt.title("Neutral Sentiments")
plt.tight_layout(pad = 0)
plt.show()